In [1]:
from src.task_vectors import TaskVector
pretrained_model = "meta-llama/Meta-Llama-3.1-8B"
finetuned_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
task_vector = TaskVector(pretrained_model, finetuned_model, device="cpu")
task_vector.save("llam3-inst-task-vector.pt")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

target_model_name = "4yo1/llama3-pre1-pre2-inst3-ds-lora3-dpo"
target_model = AutoModelForCausalLM.from_pretrained(target_model_name)
target_tokenizer = AutoTokenizer.from_pretrained(target_model_name)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
applyed_model = task_vector.apply_to(target_model, scaling_coef=0.1, handle_mismatch='use_taskvector')

Mismatched keys (replaced if using 'use_taskvector'): ['model.embed_tokens.weight', 'lm_head.weight']


RuntimeError: Error(s) in loading state_dict for LlamaForCausalLM:
	size mismatch for model.embed_tokens.weight: copying a param with shape torch.Size([128256, 4096]) from checkpoint, the shape in current model is torch.Size([149465, 4096]).
	size mismatch for lm_head.weight: copying a param with shape torch.Size([128256, 4096]) from checkpoint, the shape in current model is torch.Size([149465, 4096]).

In [ ]:
import torch
def generate_text(prompt, model, tokenizer, max_length=200, temperature=0.1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=1,
            repetition_penalty=1.0,
            do_sample=True,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [ ]:
# input_text = '입력된 영어 문장을 한국어로 번역하여 출력하세요.\n\nYoduk Story is an epic musical by North Korean defector Chung Sung-san who exposed human rights abuses in North Korean political prison camps based on true stories.\n\n' + EOS_TOKEN
input_text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system}\n<|eot_id|><|start_header_id|>user<|end_header_id|>
{user}<|eot_id|><|start_header_id|>\nassistant<|end_header_id|><|eot_id|>"""

system = "너는 번역 전문가야."
# system = "너는 친절한 챗봇이야."
user = "입력된 문장을 한국어로 번역하세요. \nYoduk Story is an epic musical by North Korean defector Chung Sung-san who exposed human rights abuses in North Korean political prison camps based on true stories."
# user = "미국 대통령은?"
input_text = input_text.format(system=system, user=user)

generated_text = generate_text(input_text, applyed_model, target_tokenizer)
response = generated_text.split("assistant")[-1].strip()
print(response)

In [ ]:
task_vector